In [2]:
import os
import time
import re
import pprint 
import json
import csv
import logging
import sys
import pandas as pd
import xlrd
import glob

from urllib.parse import parse_qsl, urljoin, urlparse


def get_video_id(the_link):
    vid = re.findall('v(?:=|%3D)([^&%?]+)', urlparse(the_link).query)
    if vid:
        return vid[0]
    the_path = urlparse(the_link).path[1:]
    if ((not the_path) or 
        the_path.startswith('user') or 
        the_path.startswith('shared') or
        the_path.startswith('playlist') or
        the_path.startswith('channel') or 
        the_path.startswith('attribution_link') or 
        the_path.startswith('results') or 
        the_path.startswith('edit') or 
        the_path.startswith('categories') or 
        the_path.startswith('c/') or 
        the_path.startswith('view_play_list')): 
            return None
    if the_path.startswith('embed/'): return the_path[len('embed/'):len('embed/')+11]#typical length of the id
    if the_path.startswith('v/'): return the_path[len('v/'):len('v/')+11]
    
    return the_path

#Input: Enter DataFrame, Return: the ID of these videos in this url
def getVideosIds(DataFrame):
    ids = []
    for i, row in DataFrame.iterrows():
        if get_video_id(row['vid_url']) != None :
            ids.append(get_video_id(row['vid_url'])+'.csv')
    return ids

# Modifying Neutral Files

In [5]:
#Now I'm just focusing on the neutral one, where I want to get ids for each one

NeutralIds = []
NeutralDataFrame = pd.read_excel('Neutral.xlsx')[['vid_url','topic']]

#They are just a lists that contain the IDs of each YouTube video by topic and by Neutral
# We first select the rows that have the topic 911 (for example), then we get their ids in a list
Neutral911 = getVideosIds(NeutralDataFrame.loc[NeutralDataFrame['topic'] == '911'])
NeutralMoonLanding =  getVideosIds(NeutralDataFrame.loc[NeutralDataFrame['topic'] == 'moonlanding'])
NeutralVaccines =  getVideosIds(NeutralDataFrame.loc[NeutralDataFrame['topic'] == 'vaccines'])
NeutralClchemtrails =  getVideosIds(NeutralDataFrame.loc[NeutralDataFrame['topic'] == 'chemtrails'])
NeutralFlatearth = getVideosIds(NeutralDataFrame.loc[NeutralDataFrame['topic'] == 'flatearth'])
 


In [7]:
# We want to get the all neutral in one (excel sheet file), and also put a column the (name of the tapic) accordingly.

# We need to use (fillna), besides removing all stop words like the (emojis ^_^)

NeutralFileNames = os.listdir('Neutral')
                
    
# Open Each file and add a topic name to all of its urls.

for fileName in NeutralFileNames:
    df = pd.read_csv('Neutral/'+fileName)
    if fileName in Neutral911:
        df["Topic"] = "911"
    elif fileName in NeutralMoonLanding:
        df["Topic"] = "moonlanding"
    elif fileName in NeutralVaccines:
        df["Topic"] = "vaccines"
    elif fileName in NeutralClchemtrails:
        df["Topic"] = "chemtrails"
    elif fileName in NeutralFlatearth:
        df["Topic"] = "flatearth"
    df["stance"] = 0
    df.to_csv('Neutral/'+fileName, index=False) 

# Modifying Anti Files

In [8]:
#Now I'm just focusing on the neutral one, where I want to get ids for each one

AntiIds = []
AntiDataFrame = pd.read_excel('Anti.xlsx')[['vid_url','topic']]

#They are just a lists that contain the IDs of each YouTube video by topic and by Neutral
Anti911 = getVideosIds(AntiDataFrame.loc[AntiDataFrame['topic'] == '911'])
AntiMoonLanding =  getVideosIds(AntiDataFrame.loc[AntiDataFrame['topic'] == 'moonlanding'])
AntiVaccines =  getVideosIds(AntiDataFrame.loc[AntiDataFrame['topic'] == 'vaccines'])
AntiChemtrails =  getVideosIds(AntiDataFrame.loc[AntiDataFrame['topic'] == 'chemtrails'])
AntiFlatearth = getVideosIds(AntiDataFrame.loc[AntiDataFrame['topic'] == 'flatearth'])
 
    
    
NeutralFileNames = os.listdir('Anti')
                
    
# Open Each file and add a topic name to all of its urls.

for fileName in NeutralFileNames:
    df = pd.read_csv('Anti/'+fileName)
    if fileName in Anti911:
        df["Topic"] = "911"
    elif fileName in AntiMoonLanding:
        df["Topic"] = "moonlanding"
    elif fileName in AntiVaccines:
        df["Topic"] = "vaccines"
    elif fileName in AntiChemtrails:
        df["Topic"] = "chemtrails"
    elif fileName in AntiFlatearth:
        df["Topic"] = "flatearth"
    df["stance"] = -1
    df.to_csv('Anti/'+fileName, index=False) 


# Modifying Pro Files

In [9]:
#Now I'm just focusing on the neutral one, where I want to get ids for each one

ProIds = []
ProDataFrame = pd.read_excel('Pro.xlsx')[['vid_url','topic']]

#They are just a lists that contain the IDs of each YouTube video by topic and by Neutral
Pro911 = getVideosIds(ProDataFrame.loc[ProDataFrame['topic'] == '911'])
ProMoonLanding =  getVideosIds(ProDataFrame.loc[ProDataFrame['topic'] == 'moonlanding'])
ProVaccines =  getVideosIds(ProDataFrame.loc[ProDataFrame['topic'] == 'vaccines'])
ProChemtrails =  getVideosIds(ProDataFrame.loc[ProDataFrame['topic'] == 'chemtrails'])
ProFlatearth = getVideosIds(ProDataFrame.loc[ProDataFrame['topic'] == 'flatearth'])
 
    
    
NeutralFileNames = os.listdir('Pro')
                
    
# Open Each file and add a topic name to all of its urls.

for fileName in NeutralFileNames:
    df = pd.read_csv('Pro/'+fileName)
    if fileName in Pro911:
        df["Topic"] = "911"
    elif fileName in ProMoonLanding:
        df["Topic"] = "moonlanding"
    elif fileName in ProVaccines:
        df["Topic"] = "vaccines"
    elif fileName in ProChemtrails:
        df["Topic"] = "chemtrails"
    elif fileName in ProFlatearth:
        df["Topic"] = "flatearth"
    df["stance"] = +1
    df.to_csv('Pro/'+fileName, index=False) 


# Combine all into one file

In [18]:

NeturalFiles = glob.glob('Neutral' + "/*.csv")
ProFiles =  glob.glob('Pro' + "/*.csv")
AntiFiles = glob.glob('Anti' + "/*.csv")
allFiles = []

for filename in NeturalFiles:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['ID'] = filename[8:-4]
    allFiles.append(df)

for filename in ProFiles:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['ID'] = filename[4:-4]
    allFiles.append(df)
    
for filename in AntiFiles:
    df = pd.read_csv(filename, index_col=None, header=0)
    df['ID'] = filename[6:-4]
    allFiles.append(df)
    
frame = pd.concat(allFiles, axis=0, ignore_index=True)

frame.to_csv('AllCommentsFinal.csv', index=False) 

df = frame

In [42]:
df = frame

In [24]:
df2 = df.loc[df['Topic'] == 'vaccines']
df2.to_excel('AllVaccinesComments.xlsx', index=False) 


df2 = df.loc[df['Topic'] == 'moonlanding']
df2.to_excel('AllMoonLandingComments.xlsx', index=False) 


df2 = df.loc[df['Topic'] == 'chemtrails']
df2.to_excel('AllchemtrailsComments.xlsx', index=False) 

df2 = df.loc[df['Topic'] == 'flatearth']
df2.to_excel('AllFlatEarthComments.xlsx', index=False) 

In [49]:
df2 = df.loc[df['Topic'] == 911]
df2.to_excel('All911Comments.xlsx', index=False) 

In [40]:
# Removing any row that has the word playlist
df = pd.read_excel('VideosDetails.xlsx')
df.loc[df['vid_url'].str.contains("playlist")]
df = df.drop(df.index[[2823,2822,2786]])
df.to_excel('VideosDetails.xlsx', index=False) 

,topic,aria-label,description,vid_title,vid_url,annotation,notes,normalized_annotation,duration,viewCount,likeCount,dislikeCount,favoriteCount,commentCount,popularity,Language
